In [ ]:
from MILWRM.MxIF import *
from MILWRM.ST import *
from MILWRM.MILWRM import *

In [ ]:
# markers in COLON MAP PCA data
allmarkers = ["ACTININ","BCATENIN","CD3D","CD4_","CD8","CD11B","CD20","CD45","CD68",
              "CGA","COLLAGEN","COX2","DAPI","ERBB2","FOXP3","GAMMAACTIN","HLAA","LYSOZYME","MUC2",
              "NAKATPASE","OLFM4","SOX9","PANCK","PCNA","PEGFR","PSTAT3","SMA","SNA","VIMENTIN"]
eliotmarkers = ["VIMENTIN","SOX9","SMA","PSTAT3","PEGFR","PCNA","PANCK","OLFM4",
                "NAKATPASE","MUC5AC","MUC2","LYSOZYME","HLAA","GAMMAACTIN","FOXP3",
                "ERBB2","COLLAGEN","CGA","CDX2","CD68","CD45","CD20","CD11B","CD8",
                "CD4_","CD3D","BCATENIN","AQP5","COX2"]

In [ ]:
def create_tissue_mask_mxif(images, markers=None, downsample_factor=1):
    """
    Creates a whole tissue mask for the given tissue image

    Parameters
    ----------
    images : list or img
        images for which the tissue mask will be created for
    markers : list
        markers required to create MxIF labeller object

    Returns
    -------
    mask is added to self.mask
    """
    if isinstance(
        images, img
    ):  # force img objects into a list if a single object is given
        images = [images]
    if markers is None:  # using all markers if list of markers is not given
        markers = images[0].ch
    # TODO : check if the images have masks already or not
    for image in images:
        # create a copy of the image
        image_cp = image.copy()
        # creating a temprory mask
        w, h, d = image_cp.img.shape
        tmp_mask = np.ones((w, h))
        # setting the mask within the image object
        image_cp.mask = tmp_mask
        # creating the mask_tl tissue labeler
        mask_tl = mxif_labeler(images=[image_cp])
        # preprocessing before running MILWRM
        mask_tl.prep_cluster_data(
            features=markers, downsample_factor=downsample_factor, sigma=2, fract=0.1
        )
        # running MILWRM with two clusters
        mask_tl.label_tissue_regions(k=2, alpha=0.05)
        # estimating centroids
        scores = mask_tl.kmeans.cluster_centers_.copy()
        mean = mask_tl.kmeans.cluster_centers_.mean(axis=0)
        sd = mask_tl.kmeans.cluster_centers_.std(axis=0)
        # z-score cluster centroid values for more reliable loadings
        z_scores = (scores - mean) / sd
        # making sure the background is set as 0
        if z_scores[0].mean() > 0:
            print(z_scores[0], "the background is set as tissue ID 1")
            where_0 = np.where(mask_tl.tissue_IDs[0] == 0.0)
            mask_tl.tissue_IDs[0][where_0] = 0.5
            where_1 = np.where(mask_tl.tissue_IDs[0] == 1.0)
            mask_tl.tissue_IDs[0][where_1] = 0.0
            where_05 = np.where(mask_tl.tissue_IDs[0] == 0.5)
            mask_tl.tissue_IDs[0][where_05] = 1.0
        # rescaling the mask
        mask_final = resize(mask_tl.tissue_IDs[0], (w, h))
        # setting the final mask
        image.mask = mask_final

---
### WD86056

In [ ]:
WD86056 = img.from_npz(file="../data/MxIF/WD86056/WD86056_stitched_downsample8.npz") ; WD86056.mask = None ; WD86056

In [ ]:
WD86056.clip()

In [ ]:
rgb = WD86056.show(channels=["CD20","BCATENIN","COLLAGEN"], RGB=True)

In [ ]:
%time create_tissue_mask_mxif(WD86056, downsample_factor=36)

In [ ]:
rgb = WD86056.show(ncols=5)

In [ ]:
%time WD86056.to_npz("../data/MxIF/WD86056/WD86056_downsample8.npz")

---
### WD85806

In [ ]:
WD85806 = img.from_npz(file="../data/MxIF/WD85806/WD85806_stitched_downsample8.npz") ; WD85806.mask = None ; WD85806

In [ ]:
WD85806.clip()

In [ ]:
rgb = WD85806.show(channels=["CD20","BCATENIN","COLLAGEN"], RGB=True)

In [ ]:
%time create_tissue_mask_mxif(WD85806, downsample_factor=36)

In [ ]:
rgb = WD85806.show(ncols=5)

In [ ]:
%time WD85806.to_npz("../data/MxIF/WD85806/WD85806_downsample8.npz")

---
### WD87480

In [ ]:
WD87480 = img.from_npz(file="../data/MxIF/WD87480/WD87480_stitched_downsample8.npz") ; WD87480.mask = None ; WD87480

In [ ]:
WD87480.clip()

In [ ]:
rgb = WD87480.show(channels=["CD20","BCATENIN","COLLAGEN"], RGB=True)

In [ ]:
%time create_tissue_mask_mxif(WD87480, downsample_factor=16)

In [ ]:
rgb = WD87480.show(ncols=5)

In [ ]:
%time WD87480.to_npz("../data/MxIF/WD87480/WD87480_downsample8.npz")

---
### WD87482

In [ ]:
WD87482 = img.from_npz(file="../data/MxIF/WD87482/WD87482_stitched_downsample8.npz") ; WD87482.mask = None ; WD87482

In [ ]:
WD87482.clip()

In [ ]:
rgb = WD87482.show(channels=["CD20","BCATENIN","COLLAGEN"], RGB=True)

In [ ]:
%time create_tissue_mask_mxif(WD87482, downsample_factor=32)

In [ ]:
rgb = WD87482.show(ncols=5)

In [ ]:
%time WD87482.to_npz("../data/MxIF/WD87482/WD87482_downsample8.npz")

---
### vTMA_001

In [ ]:
%%time
vTMA1 = img.from_tiffs(
    tiffdir = "../data/MxIF/VISIUM_TMA/AFRemoved/",
    common_strings = ["AFRemoved_pyr16_region_001.tif"],
    channels = eliotmarkers,
    #mask = "01_TISSUE_MASK.tif",
)

In [ ]:
vTMA1

In [ ]:
%%time
vTMA1.downsample(fact=8)
vTMA1.clip()
vTMA1

In [ ]:
rgb = vTMA1.show()

In [ ]:
%time vTMA1.to_npz("../data/MxIF/vTMA_001/vTMA_001_downsample8.npz")

In [ ]:
%time create_tissue_mask_mxif(vTMA1, downsample_factor=8)

In [ ]:
rgb = vTMA1.show()

In [ ]:
%time vTMA1.to_npz("../data/MxIF/vTMA_001/vTMA_001_downsample8.npz")

---
### vTMA_002

In [ ]:
%%time
vTMA1 = img.from_tiffs(
    tiffdir = "../data/MxIF/VISIUM_TMA/AFRemoved/",
    common_strings = ["AFRemoved_pyr16_region_002.tif"],
    channels = eliotmarkers,
    #mask = "01_TISSUE_MASK.tif",
)

In [ ]:
vTMA1

In [ ]:
%%time
vTMA1.downsample(fact=8)
vTMA1.clip()
vTMA1

In [ ]:
rgb = vTMA1.show()

In [ ]:
%time vTMA1.to_npz("../data/MxIF/vTMA_002/vTMA_002_downsample8.npz")

In [ ]:
%time create_tissue_mask_mxif(vTMA1, downsample_factor=8)

In [ ]:
rgb = vTMA1.show()

In [ ]:
%time vTMA1.to_npz("../data/MxIF/vTMA_002/vTMA_002_downsample8.npz")

---
### MPP-00092-A1

In [ ]:
%%time
MPP_00092_A1 = img.from_tiffs(
    tiffdir = "../data/MxIF/MPP_0092_A1/AFRemoved/",
    common_strings = ["stitched_downsample8.tif"],
    channels = eliotmarkers,
)

In [ ]:
MPP_00092_A1

In [ ]:
%%time
MPP_00092_A1.clip()
MPP_00092_A1

In [ ]:
rgb = MPP_00092_A1.show(ncols=5)

In [ ]:
%time MPP_00092_A1.to_npz("../data/MxIF/MPP_00092_A1/MPP_00092_A1_downsample8.npz")

In [ ]:
%time create_tissue_mask_mxif(MPP_00092_A1, downsample_factor=8)

In [ ]:
rgb = MPP_00092_A1.show(ncols=5)

In [ ]:
%time MPP_00092_A1.to_npz("../data/MxIF/MPP_00092_A1/MPP_00092_A1_downsample8.npz")

---
### MPP-00274-A1

In [ ]:
%%time
MPP_00274_A1 = img.from_tiffs(
    tiffdir = "../data/MxIF/MPP_00274_A1/AFRemoved/",
    common_strings = ["stitched_downsample8.tif"],
    channels = eliotmarkers,
)

In [ ]:
MPP_00274_A1

In [ ]:
MPP_00274_A1.clip()

In [ ]:
rgb = MPP_00274_A1.show(ncols=5)

In [ ]:
%time MPP_00274_A1.to_npz("../data/MxIF/MPP_00274_A1/MPP_00274_A1_downsample8.npz")

In [ ]:
%time create_tissue_mask_mxif(MPP_00274_A1, downsample_factor=8)

In [ ]:
rgb = MPP_00274_A1.show(ncols=5)

In [ ]:
%time MPP_00274_A1.to_npz("../data/MxIF/MPP_00274_A1/MPP_00274_A1_downsample8.npz")

---
### MPP-00274-B1

In [ ]:
%%time
MPP_00274_B1 = img.from_tiffs(
    tiffdir = "../data/MxIF/MPP_00274_B1/AFRemoved/",
    common_strings = ["stitched_downsample8.tif"],
    channels = eliotmarkers,
)

In [ ]:
MPP_00274_B1

In [ ]:
MPP_00274_B1.clip()

In [ ]:
rgb = MPP_00274_B1.show(ncols=5)

In [ ]:
%time create_tissue_mask_mxif(MPP_00274_B1, downsample_factor=8)

In [ ]:
rgb = MPP_00274_B1.show(ncols=5)

In [ ]:
%time MPP_00274_B1.to_npz("../data/MxIF/MPP_00274_B1/MPP_00274_B1_downsample8.npz")

---
### WD87478

In [ ]:
%%time
WD87478 = img.from_tiffs(
    tiffdir = "../data/MxIF/WD87478/AFRemoved/",
    common_strings = ["stitched_downsample8.tif"],
    channels = eliotmarkers,
)
WD87478

In [ ]:
WD87478.clip()

In [ ]:
rgb = WD87478.show(ncols=5)

In [ ]:
%time create_tissue_mask_mxif(WD87478, downsample_factor=8)

In [ ]:
rgb = WD87478.show(ncols=5)

In [ ]:
%time WD87478.to_npz("../data/MxIF/WD87478/WD87478_downsample8.npz")

---
### WD33468

In [ ]:
%%time
WD33468 = img.from_tiffs(
    tiffdir = "../data/MxIF/WD33468/AFRemoved/",
    common_strings = ["stitched_downsample8.tif"],
    channels = eliotmarkers,
)
WD33468

In [ ]:
WD33468.clip()

In [ ]:
rgb = WD33468.show(channels=["CD20","BCATENIN","COLLAGEN"], RGB=True)

In [ ]:
%time create_tissue_mask_mxif(WD33468, downsample_factor=16)

In [ ]:
rgb = WD33468.show(ncols=5)

In [ ]:
%time WD33468.to_npz("../data/MxIF/WD33468/WD33468_downsample8.npz")

---
### WD33475

In [ ]:
%%time
WD33475 = img.from_tiffs(
    tiffdir = "../data/MxIF/WD33475/AFRemoved/",
    common_strings = ["stitched_downsample8.tif"],
    channels = eliotmarkers,
)
WD33475

In [ ]:
WD33475.clip()

In [ ]:
rgb = WD33475.show(channels=["CD20","BCATENIN","COLLAGEN"], RGB=True)

In [ ]:
%time create_tissue_mask_mxif(WD33475, downsample_factor=16)

In [ ]:
rgb = WD33475.show(ncols=5)

In [ ]:
%time WD33475.to_npz("../data/MxIF/WD33475/WD33475_downsample8.npz")

---
### WD33476

In [ ]:
%%time
WD33476 = img.from_tiffs(
    tiffdir = "../data/MxIF/WD33476/AFRemoved/",
    common_strings = ["stitched_downsample8.tif"],
    channels = eliotmarkers,
)
WD33476

In [ ]:
WD33476.clip()

In [ ]:
rgb = WD33476.show(channels=["CD20","BCATENIN","COLLAGEN"], RGB=True)

In [ ]:
%time create_tissue_mask_mxif(WD33476, downsample_factor=16)

In [ ]:
rgb = WD33476.show(ncols=5)

In [ ]:
%time WD33476.to_npz("../data/MxIF/WD33476/WD33476_downsample8.npz")

---
### WD33479

In [ ]:
%%time
WD33479 = img.from_tiffs(
    tiffdir = "../data/MxIF/WD33479/AFRemoved/",
    common_strings = ["stitched_downsample8.tif"],
    channels = eliotmarkers,
)
WD33479

In [ ]:
WD33479.clip()

In [ ]:
rgb = WD33479.show(channels=["CD20","BCATENIN","COLLAGEN"], RGB=True)

In [ ]:
%time create_tissue_mask_mxif(WD33479, downsample_factor=16)

In [ ]:
rgb = WD33479.show(ncols=5)

In [ ]:
%time WD33479.to_npz("../data/MxIF/WD33479/WD33479_downsample8.npz")